<a href="https://colab.research.google.com/github/jhy9968/ECE6179_project/blob/main/Create_refined_unlabelled_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torchvision.datasets import STL10 as STL10
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torchvision
from google.colab import drive
drive.mount('/content/drive/')

####### CHANGE TO APPROPRIATE DIRECTORY TO STORE DATASET
root_dir = "/content/drive/Shareddrives/ECE6179_project/"
dataset_dir = root_dir + "CNN-VAE/data/"
#For MonARCH
# dataset_dir = "/mnt/lustre/projects/ds19/SHARED"

#All images are 3x96x96
image_size = 96

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
transform_unlabelled = transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [8]:
unlabelled_set = STL10(dataset_dir, split='unlabeled', transform=transform_unlabelled, download=True)

Files already downloaded and verified


In [20]:
from torch.utils.data import DataLoader

batch_size = 64

unlabelled_loader = DataLoader(unlabelled_set, shuffle=False, batch_size=batch_size)

print('Size of unlabelled loader:', len(unlabelled_loader))
print('Batch size:', batch_size)

Size of unlabelled loader: 1563
Batch size: 64


In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(in_features=512, out_features=10, bias=True)
model = model.to(device)
model.load_state_dict(torch.load(root_dir+'Baseline_resnet18/models/trained_baseline18'))

softmax = torch.nn.Softmax()

cuda:0


<All keys matched successfully>

In [29]:
out_list = []

for images, labels in unlabelled_loader:
  images = images.to(device)
  with torch.no_grad():
    out = model(images)
    out_list.append(softmax(out))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


In [59]:
all_out = torch.cat(out_list, dim=0)
print(all_out.shape)

torch.Size([100000, 10])


In [66]:
import numpy as np

refined_images = []
K = 1000

for i in range(10): 
  col = all_out[:,i].cpu().detach().numpy()
  ind = np.argpartition(col, -K)[-K:]
  for j in ind:
    refined_images.append(unlabelled_set[j][0])

refined_images = torch.stack(refined_images)
print(refined_images.shape)

torch.Size([10000, 3, 96, 96])


In [67]:
torch.save(refined_images, dataset_dir+'refined_images.pt')



> Create dataset and dataloader for the refined data



In [76]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class Refined_Unlabelled_Dataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.images = torch.load(img_dir)
        self.transform = transform

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):
        image = self.images[i]
        label = -1
        if self.transform:
            image = self.transform(image)
        return image, label

In [77]:
root_dir = "/content/drive/Shareddrives/ECE6179_project/"
dataset_dir = root_dir + "CNN-VAE/data/"
refined_unlabelled_set = Refined_Unlabelled_Dataset(img_dir=dataset_dir+'refined_images.pt')

In [78]:
batch_size = 32
refined_unlabelled_loader = DataLoader(refined_unlabelled_set, batch_size=batch_size, shuffle=True)